In [1]:
import cv2
import numpy as np
from keras.models import load_model

## Load the model

In [2]:
PATH = "emotion_detector.hdf5"

model = load_model(PATH)

2023-02-08 15:16:56.572692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 15:16:56.576216: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_18 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_17 (Conv2D)          (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_19 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                      

### Load Haar Cascade OpenCV face detector

In [5]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Live detection

In [9]:
cap = cv2.VideoCapture(-1)

[ WARN:0@151.858] global /opt/conda/conda-bld/opencv-suite_1664548337286/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@152.114] global /opt/conda/conda-bld/opencv-suite_1664548337286/work/modules/videoio/src/cap_v4l.cpp (814) requestBuffers VIDEOIO(V4L2:/dev/video0): failed VIDIOC_REQBUFS: errno=19 (No such device)


In [10]:
cap.set(cv2.CAP_PROP_BRIGHTNESS, 30)
cap.set(cv2.CAP_PROP_CONTRAST, 50)
cap.set(cv2.CAP_PROP_SATURATION, 60)

True

In [11]:
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Change frame color to gray
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Face detection with OpenCV haar cascade
    faces_detected = face_haar_cascade.detectMultiScale(gray)
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(gray,(x,y), (x+w,y+h), (255,0,0))

        roi_gray=gray[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))
        image_pixels = np.expand_dims(roi_gray, axis = 0)

    # Prediction of emotion
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])

    emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    emotion_prediction = emotion_detection[max_index]

    cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)  

    resized_image = cv2.resize(frame, (1000, 700))
    cv2.imshow('Emotion',resized_image)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 98ms/step
